In [1]:
import pandas as pd
import numpy as np
import covidcast
from datetime import date, datetime, timedelta
import os

In [2]:
os.chdir('/Users/hongrudu/Desktop/COVID_Vaccination_rate/data/raw_data')

In [3]:
def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta

In [4]:
def create_relative(df, col):
    df_ref = df[['Week_Mon', col]].groupby(['Week_Mon']).mean().reset_index()
    df_ref = df_ref.rename(columns = {col : 'ref'})
    df = df.merge(df_ref, left_on = ['Week_Mon'], right_on = ['Week_Mon'])
    df['Relative_' + col] = df[col]/df['ref']
    df = df.drop(columns = ['ref'])
    return df

In [6]:
cases = pd.read_pickle('Weekly_cases.pkl')
hospitalized = pd.read_pickle('Weekly_hospitalization.pkl')
df = hospitalized.merge(cases)
#### Smooth data
for fips in df['FIPS'].unique():
    df.loc[df[df['FIPS'] == fips].index, ['Cases']] =\
    df[df['FIPS'] == fips][['Cases']].rolling(window = 3).mean()
df = df.dropna().reset_index().drop(columns = ['index'])

for fips in df['FIPS'].unique():
    df.loc[df[df['FIPS'] == fips].index, ['Cases_lag1']] =\
    df[df['FIPS'] == fips]['Cases'].shift(2)
df = df.dropna().reset_index().drop(columns = ['index'])
df['IHR_lag1'] = df['hospitalized']/df['Cases_lag1']
for fips in df['FIPS'].unique():
    df.loc[df[df['FIPS'] == fips].index, ['IHR_lag1']] =\
    df[df['FIPS'] == fips][['IHR_lag1']].rolling(window = 3).mean()
    
pop = pd.read_pickle('State_population.pkl')
df = pop.merge(df)

In [7]:
#### Vaccine Data
vaccine = pd.read_pickle('Weekly_vaccination.pkl')
df = df.merge(vaccine)
for i in vaccine.columns[2:]:
    df[i] = df[i]/df['Population']
for i in vaccine.columns[2:]:
    df[i] = df[i]/7

In [8]:
for fips in ['35', '42']:
    df.loc[df[df['FIPS'] == fips].index, ['People_fully_vaccinated']] =\
    df[df['FIPS'] == fips][['People_fully_vaccinated']].rolling(window = 3).mean()
    
##### Policy data
policy = pd.read_pickle('Weekly_policy.pkl')
df = df.merge(policy)

###### Testing data
test = pd.read_pickle('Weekly_testing.pkl')
test.Weekly_testing = test.Weekly_testing.mask(test.Weekly_testing.lt(0),np.nan)
test['Weekly_testing'] = test['Weekly_testing'].interpolate(method='linear')
df = df.merge(test)
for i in test.columns[2:]:
    df[i] = df[i]/df['Population']
for fips in df['FIPS'].unique():
    df.loc[df[df['FIPS'] == fips].index, ['Weekly_testing']] =\
    df[df['FIPS'] == fips][['Weekly_testing']].rolling(window = 3).mean()

In [10]:
#### Past infections
past_inf = pd.read_pickle('Weekly_past_infection.pkl')
df = df.merge(past_inf)
for i in past_inf.columns[2:]:
    df[i] = df[i]/df['Population']

In [11]:
df['CIR'] = df['Cases']/df['Population']

In [12]:

##### Add lags

for col in ['People_at_least_one_dose', 'People_fully_vaccinated',
       'Total_additional_doses', 'GovernmentResponseIndex_Average',
       'Weekly_testing']:
    for fips in df['FIPS'].unique():
         df.loc[df[df['FIPS'] == fips].index, col + '_lag1'] =\
         df[df['FIPS'] == fips][col].shift(1)

for col in ['Past_16_week_infections', 'Past_12_week_infections',
       'Past_20_week_infections', 'Past_24_week_infections']:
    for fips in df['FIPS'].unique():
         df.loc[df[df['FIPS'] == fips].index, col + '_lag3'] =\
         df[df['FIPS'] == fips][col].shift(3)

In [13]:
df = df[~df['FIPS'].isin(['33', '54'])].reset_index().drop(columns = ['index'])

In [14]:
df_selected = df[['FIPS', 'Week_Mon',  'CIR','People_at_least_one_dose_lag1', 'People_fully_vaccinated_lag1',
       'Total_additional_doses_lag1', 'GovernmentResponseIndex_Average_lag1',
       'Weekly_testing_lag1', 'Past_16_week_infections_lag3',
       'Past_12_week_infections_lag3', 'Past_20_week_infections_lag3',
       'Past_24_week_infections_lag3']]

In [15]:
for col in df_selected.columns[2:]:
    df_selected = create_relative(df_selected, col)

In [16]:
mobility = pd.read_pickle('Weekly_activity_level.pkl')
mobility['Week_Mon'] = pd.to_datetime(mobility['Week_Mon'])
for fips in mobility['FIPS'].unique():
    mobility.iloc[mobility[mobility['FIPS'] == fips].index,2:] =\
    mobility[mobility['FIPS'] == fips].iloc[:, 2:].rolling(window = 3).mean()

In [17]:
df_selected = df_selected.merge(mobility, how = 'left')
for col in mobility.columns[2:]:
    for fips in df_selected['FIPS'].unique():
         df_selected.loc[df_selected[df_selected['FIPS'] == fips].index, col + '_lag1'] =\
         df_selected[df_selected['FIPS'] == fips][col].shift(1)

In [18]:
df_selected = df_selected[['FIPS', 'Week_Mon', 'Relative_CIR',
                           'Relative_People_at_least_one_dose_lag1',
       'Relative_People_fully_vaccinated_lag1',
       'Relative_Total_additional_doses_lag1',
       'Relative_GovernmentResponseIndex_Average_lag1',
       'Relative_Weekly_testing_lag1', 'Relative_Past_16_week_infections_lag3',
       'Relative_Past_12_week_infections_lag3',
       'Relative_Past_20_week_infections_lag3',
       'Relative_Past_24_week_infections_lag3',
       'limited_service_restaurant_lag1', 'elementary_school_lag1',
       'other_general_store_lag1', 'gas_station_lag1', 'gym_lag1',
       'cafes_snack_bar_lag1', 'religious_organization_lag1', 'parks_lag1',
       'hardware_store_lag1', 'department_store_lag1', 'child_day_care_lag1',
       'office_of_physician_lag1', 'pharmacies_lag1',
       'sporting_goods_store_lag1', 'automotive_store_lag1',
       'used_merchandise_store_lag1', 'convenience_store_lag1',
       'grocery_store_lag1', 'full_service_restaurant_lag1', 'hotel_lag1',
       'University_lag1']]

In [19]:
for col in ['limited_service_restaurant_lag1', 'elementary_school_lag1',
       'other_general_store_lag1', 'gas_station_lag1', 'gym_lag1',
       'cafes_snack_bar_lag1', 'religious_organization_lag1', 'parks_lag1',
       'hardware_store_lag1', 'department_store_lag1', 'child_day_care_lag1',
       'office_of_physician_lag1', 'pharmacies_lag1',
       'sporting_goods_store_lag1', 'automotive_store_lag1',
       'used_merchandise_store_lag1', 'convenience_store_lag1',
       'grocery_store_lag1', 'full_service_restaurant_lag1', 'hotel_lag1',
       'University_lag1']:
    df_selected = create_relative(df_selected, col)

In [20]:
df_selected = df_selected[['FIPS', 'Week_Mon', 'Relative_CIR', 
                           'Relative_People_at_least_one_dose_lag1',
       'Relative_People_fully_vaccinated_lag1',
       'Relative_Total_additional_doses_lag1',
       'Relative_GovernmentResponseIndex_Average_lag1',
       'Relative_Weekly_testing_lag1', 'Relative_Past_16_week_infections_lag3',
       'Relative_Past_12_week_infections_lag3',
       'Relative_Past_20_week_infections_lag3',
       'Relative_Past_24_week_infections_lag3',
       'Relative_limited_service_restaurant_lag1',
       'Relative_elementary_school_lag1', 'Relative_other_general_store_lag1',
       'Relative_gas_station_lag1', 'Relative_gym_lag1',
       'Relative_cafes_snack_bar_lag1', 'Relative_religious_organization_lag1',
       'Relative_parks_lag1', 'Relative_hardware_store_lag1',
       'Relative_department_store_lag1', 'Relative_child_day_care_lag1',
       'Relative_office_of_physician_lag1', 'Relative_pharmacies_lag1',
       'Relative_sporting_goods_store_lag1', 'Relative_automotive_store_lag1',
       'Relative_used_merchandise_store_lag1',
       'Relative_convenience_store_lag1', 'Relative_grocery_store_lag1',
       'Relative_full_service_restaurant_lag1', 'Relative_hotel_lag1',
       'Relative_University_lag1']]

In [21]:
genomic = pd.read_pickle('Weekly_genomic.pkl')
genomic['Week_Mon'] = pd.to_datetime(genomic['Week_Mon'])
df_selected = df_selected.merge(genomic)
df_selected['Dominant_variant'] = df_selected['Dominant_variant'].replace({'Alpha' : 'Others'})

In [22]:
static = pd.read_pickle('Static_data.pkl')
df_selected = df_selected.merge(static)

In [23]:
df_selected = df_selected[df_selected['Week_Mon']>='2021-03-01'].reset_index().drop(columns = ['index'])
df_selected = df_selected[df_selected['Week_Mon']<='2022-03-01'].reset_index().drop(columns = ['index'])
df_selected['medicaid_spending'] = df_selected['medicaid_spending']/1000

In [24]:
df_selected = df_selected[['FIPS', 'Week_Mon', 'Relative_CIR',
       'Relative_People_at_least_one_dose_lag1',
       'Relative_People_fully_vaccinated_lag1',
       'Relative_Total_additional_doses_lag1',
       'Relative_GovernmentResponseIndex_Average_lag1',
       'Relative_Weekly_testing_lag1', 'Relative_Past_16_week_infections_lag3',
       'Relative_Past_12_week_infections_lag3',
       'Relative_Past_20_week_infections_lag3',
       'Relative_Past_24_week_infections_lag3', 'Relative_University_lag1',
       'Relative_full_service_restaurant_lag1','Relative_gas_station_lag1',
       'Relative_gym_lag1',
       'Relative_religious_organization_lag1',
       'Relative_office_of_physician_lag1','over_65',
       'Black', 'medicaid_spending', 'health_care_spending', 'poverty_rate',
       'social_vulnerability_index',
       'Healthcare Access and Quality Index', 'rep_voters',
       'Adults_at_high_risk', 'Dominant_variant']]

In [25]:
df_selected = df_selected.rename(columns = {'Relative_CIR' : 'RCIR'})
df_selected = df_selected.rename(columns = {'Relative_People_at_least_one_dose_lag1' : 'RPVR_lag1',
                             'Relative_People_fully_vaccinated_lag1' : 'RCPSR_lag1',
                             'Relative_Total_additional_doses_lag1' : 'RBVR_lag1',
                             'Relative_GovernmentResponseIndex_Average_lag1' : 'RGP_lag1',
                             'Relative_Weekly_testing_lag1' : 'RWTR_lag1',
                             'Relative_Past_12_week_infections_lag3' : 'RPI_12_Weeks_lag3',
                             'Relative_Past_16_week_infections_lag3' : 'RPI_16_Weeks_lag3',
                             'Relative_Past_20_week_infections_lag3' : 'RPI_20_Weeks_lag3',
                             'Relative_Past_24_week_infections_lag3' : 'RPI_24_Weeks_lag3',
                             'Relative_University_lag1' : 'RUV_lag1',
                            'Relative_full_service_restaurant_lag1' : 'RFSRV_lag1',
                             'Relative_gas_station_lag1' : 'RGSV_lag1',
                             'Relative_gym_lag1' : 'RGV_lag1', 
                             'Relative_religious_organization_lag1' : 'RROV_lag1',
                             'Relative_office_of_physician_lag1' : 'ROPV_lag1',
                             'Healthcare Access and Quality Index' : 'HAQI',
                             'social_vulnerability_index' : 'SVI'})

In [26]:
df_selected.to_csv('/Users/hongrudu/Desktop/COVID_Vaccination_rate/data/processed_data/CIR_input.csv')